## Preperation: load dependencies

In [ ]:
//import $ivy.`com.typesafe.akka::akka-http-spray-json:10.2.4`
//import $ivy.`com.typesafe.slick::slick:3.3.3`
//import $ivy.`com.typesafe.slick::slick-hikaricp:3.3.3`
//import $ivy.`com.github.tminglei::slick-pg:0.19.5`
//import $ivy.`com.github.tminglei::slick-pg_spray-json:0.19.5`
//import $ivy.`org.postgresql:postgresql:42.2.19`
//import $ivy.`ch.qos.logback:logback-classic:1.2.3`

// Load fat JAR
interp.load.cp(os.Path("/home/jovyan/jar/masterthesis-khipu-assembly-0.1.jar"))

In [ ]:
// Smile

import $ivy.`com.github.haifengl::smile-scala:2.6.0`

import scala.language.postfixOps
import org.apache.commons.csv.CSVFormat
import smile._
import smile.util._
import smile.math._
import smile.math.MathEx._
import smile.math.kernel._
import smile.math.matrix._
import smile.math.matrix.Matrix._
import smile.math.rbf._
import smile.stat.distribution._
import smile.data._
import smile.data.formula._
import smile.data.measure._
import smile.data.`type`._
import smile.json._
import smile.interpolation._
import smile.validation._
import smile.association._
import smile.base.cart.SplitRule
import smile.base.mlp._
import smile.base.rbf.RBF
import smile.classification._
import smile.regression._
import smile.feature._
import smile.clustering._
import smile.vq._
import smile.manifold._
import smile.mds._
import smile.sequence._
import smile.projection._
import smile.nlp._
import smile.wavelet._
// Plotting
import smile.plot.swing._
import smile.plot.vega._
import smile.plot.show
import smile.plot.Render._

## Get data from DB

In [ ]:
import scala.concurrent.ExecutionContext.Implicits.global
import scala.concurrent.Future
import scala.concurrent.duration._
import scala.concurrent.{ Await, Future }
import scala.language.postfixOps

import de.tmrdlt.database.workflowlist.WorkflowListDB
import de.tmrdlt.database.event.EventDB


val workflowListDB = new WorkflowListDB()
val eventDB = new EventDB()

val workflowLists =  Await.result(workflowListDB.getWorkflowLists, 2 seconds)
val events =  Await.result(eventDB.getEvents, 2 seconds)

# Trello
## Size of boards and columns over time

In [ ]:
import de.tmrdlt.models._
import de.tmrdlt.database.workflowlist._
import java.time.LocalDate
import scala.collection.mutable.ListBuffer

val boards = workflowLists
    .filter(_.listType == WorkflowListType.BOARD)
    .filter(_.dataSource == WorkflowListDataSource.Trello)
print(boards.length)

In [5]:
def plotDataForBoard(board: WorkflowList) = {
    val data = new ListBuffer[(Int, LocalDate)]
    data += ((0, board.createdAt.toLocalDate))
    
    events
        .filter(e => (e.eventType == "createCard" || e.eventType == "deleteCard") 
               && e.boardApiId == Some(board.apiId))
        .sortBy(_.date)
        .foreach { e => 
            val count = if (e.eventType == "createCard") data.last._1 + 1 else data.last._1 - 1
            data += ((count, e.date.toLocalDate))
        }
    
    val smileData = data.toList.map(d => JsObject("date" -> JsDate(d._2), "count" -> JsInt(d._1)))
    val chart = VegaLite.view()
        .mark(JsObject("type" -> "area", "line" -> true, "point" -> true))
        .width(800)
        .height(200)
        .x(field = "date", `type` = "temporal")
        .y(field = "count", `type` = "quantitative")
        .data(smileData)
        .title(s"Size of the board '${board.title}' over time")
    show(chart)
    
    val columns = workflowLists
    .filter(_.listType == WorkflowListType.LIST)
    .filter(_.parentId == Some(board.id))
    .sortBy(_.position)
    val columnData = 
    columns.map { column =>
        val data = new ListBuffer[(Int, LocalDate, String, Int)]
        data += ((0, column.createdAt.toLocalDate, column.title, column.position.toInt))
        
        events
            .filter(e => e.parentApiId == Some(column.apiId) ||
                         e.oldParentApiId == Some(column.apiId) ||
                         e.newParentApiId == Some(column.apiId))
            .sortBy(_.date)
            .foreach { e => 
                val count = 
                    if (e.eventType == "createCard") {
                        data.last._1 + 1
                    } else if (e.eventType == "deleteCard") {
                        data.last._1 - 1  
                    } else {
                        if (column.apiId == e.oldParentApiId.get) {
                            data.last._1 - 1
                        } else {
                            data.last._1 + 1
                        }
                    } 
                data += ((count, e.date.toLocalDate, column.title, column.position.toInt))
            }
        data
    }
    
    columnData.foreach { data =>
        val smileData = data.toList.map(d => JsObject("date" -> JsDate(d._2), "count" -> JsInt(d._1)))
        val chart = VegaLite.view()
            .mark(JsObject("type" -> "area", "line" -> true, "point" -> true))
            .width(800)
            .height(200)
            .x(field = "date", `type` = "temporal")
            .y(field = "count", `type` = "quantitative")
            .data(smileData)
            .title(s"Size of column '${data(0)._3}' over time")
        show(chart)
    }
    // Plot as Facet
    //val columnSmileData = columnData.flatten.toList.map(d => JsObject("card_count" -> JsInt(d._1), "date" -> JsDate(d._2), "column_name" -> JsString(d._3), "position" -> JsInt(d._4)))
    //val chart2 = VegaLite.facet(columnSmileData)
    //    .row(field = "column_name", `type` = "nominal")
    //    .mark(JsObject("type" -> "area", "line" -> true, "point" -> true))
    //    .width(800)
    //    .height(150)
    //    .x(field = "date", `type` = "temporal")
    //    .y(field = "card_count", `type` = "quantitative")
    //    .title(s"Size of the columns of board '${board.title}' over time")
    //show(chart2)
}

defined function plotDataForBoard

In [6]:
plotDataForBoard(boards(0))

: 